In [ ]:
# 导入合适的库
import math
import torch
import gpytorch
import pyro
from pyro.infer.mcmc import NUTS, MCMC, HMC
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# 导入数据
params = np.loadtxt('0823paramsnp.txt')
edpResults = np.loadtxt('0823edpResult.txt')

In [ ]:
# 检查params数据是否正确 [T1, mb, kesi, PGA, PGV, PGD, Sd, Sv, Sa]
print(params.shape)
params

In [ ]:
# 检查edpResults数据是否正确 [IDR1_MAX, IDR2_MAX, IDR3_MAX, amax0, amax1, amax2, amax3, residual_idr]
print(edpResults.shape)
edpResults = np.log(edpResults)

In [ ]:
from sklearn.preprocessing import StandardScaler
# 创建 StandardScaler 实例
scaler = StandardScaler()
# 假设 X 是输入特征数据
# 在训练集上拟合（计算均值和方差），并对数据进行标准化
X_train_scaled = scaler.fit_transform(params[:1600])
# 在测试集上使用相同的标准化器进行标准化
X_test_scaled = scaler.transform(params[1600:])

In [ ]:
# 数据抓换成torch格式
n = 4
train_x = torch.from_numpy(X_train_scaled).to(torch.float)
train_y = torch.from_numpy(edpResults[:1600, n]).to(torch.float)

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [ ]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
num_samples = 2 if smoke_test else 100
warmup_steps = 2 if smoke_test else 100


from gpytorch.priors import LogNormalPrior, NormalPrior, UniformPrior
# Use a positive constraint instead of usual GreaterThan(1e-4) so that LogNormal has support over full range.
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y, likelihood)

model.mean_module.register_prior("mean_prior", UniformPrior(-1, 1), "constant")
model.covar_module.base_kernel.register_prior("lengthscale_prior", UniformPrior(0.01, 0.5), "lengthscale")
model.covar_module.register_prior("outputscale_prior", UniformPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", UniformPrior(0.01, 0.5), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def pyro_model(x, y):
    with gpytorch.settings.fast_computations(False, False, False):
        sampled_model = model.pyro_sample_from_prior()
        output = sampled_model.likelihood(sampled_model(x))
        pyro.sample("obs", output, obs=y)
    return y

nuts_kernel = NUTS(pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps, disable_progbar=smoke_test)
mcmc_run.run(train_x, train_y)

In [ ]:
model.pyro_load_from_samples(mcmc_run.get_samples())

In [ ]:
model.eval()
test_x = torch.from_numpy(X_test_scaled).to(torch.float)
output = model(test_x)

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.from_numpy(X_test_scaled).to(torch.float)
    observed_pred_test = likelihood(model(test_x))

In [ ]:
from sklearn.metrics import r2_score
r_squared_sklearn_test = r2_score(edpResults[1600:, n], observed_pred_test.mean.numpy())
print(r_squared_sklearn_test)

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # test_x = torch.from_numpy(params[1600:]).to(torch.float)
    observed_pred_train = likelihood(model(train_x))

In [ ]:
from sklearn.metrics import r2_score
r_squared_sklearn_train = r2_score(edpResults[:1600, n], observed_pred_train.mean.numpy())
print(r_squared_sklearn_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 生成示例数据（模拟预测值和观测值）
np.random.seed(0)
# observed_values = edpResults[:1600, 0]
# predicted_values = observed_pred_train.mean.numpy()
observed_values = edpResults[1600:, n]
predicted_values = observed_pred_test.mean.numpy()
# 绘制散点图和对角线
a = 1
plt.figure(figsize=(4,4))
plt.scatter(predicted_values, observed_values, color='blue', label='Data')
plt.plot([0, a], [0, a], color='red', linestyle='--', label='Diagonal Line')
plt.xlabel('Predicted Values')
plt.ylabel('Observed Values')
plt.xlim([0, 1])
plt.ylim([0, 1])
# plt.title('QQ Plot with Diagonal Line')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 生成示例数据（模拟预测值和观测值）
np.random.seed(0)
# observed_values = edpResults[:1600, 0]
# predicted_values = observed_pred_train.mean.numpy()
observed_values = edpResults[1600:, n]
predicted_values = observed_pred_test.mean.numpy()
# 绘制散点图和对角线
a = 0.02
plt.figure(figsize=(4,4))
plt.scatter(predicted_values, observed_values, color='blue', label='Data')
plt.plot([0, a], [0, a], color='red', linestyle='--', label='Diagonal Line')
plt.xlabel('Predicted Values')
plt.ylabel('Observed Values')
plt.xlim([0, 0.02])
plt.ylim([0, 0.02])
# plt.title('QQ Plot with Diagonal Line')
plt.legend()
plt.grid(True)
plt.show()